In [1]:
import numpy as np
import matplotlib.pylab as plt
import math
import pandas as pd
import seaborn as sns
sns.set()
from matplotlib import rcParams
rcParams['figure.figsize'] = 11.7,8.27

In [36]:
class RLS_2:
    def __init__(self, num_vars, lam, delta):
        '''
        num_vars: Degree of polinomial
        lam: Forgetting factor, usually very close to 1
        delta: Initation value -> ! Bad initation needs more itteration to reach same accuracy
        '''
        self.num_vars = num_vars
        self.P = delta*np.matrix(np.identity(self.num_vars))

        # Weigths/Coefficent of the system
        self.w = np.matrix(np.zeros(self.num_vars))
        self.w = self.w.reshape(self.w.shape[1],1)

        # Kalman Gain Factor
        self.g = np.matrix(np.zeros(num_vars))
        self.g = np.reshape(self.g,(num_vars,1))
        
        # Variables needed for add_obs
        self.lam_inv = lam**(-1)
        
        # A priori error
        self.a_priori_error = 0
        
        # Count of number of observations added
        self.num_obs = 0

    def add_obs(self, x, t):
        '''
        Expected value is t, add the new observation as t.
        t is noisy output of the some linear system. Input of the RLS.

        Task is to identify a system, by determining coefficents,
        that outputs a value which is closest to t.

        x is a column vector as a numpy matrix  |   (new inputs to the system)
        t is a real scalar                      |   (expected output to update weigths)
        '''            

        self.g = self.lam_inv*self.P*x/(1+self.lam_inv*(x.T*self.P*x))
        self.P = self.P*self.lam_inv - self.g*(x.T*self.P)
        self.w = self.w + self.g*(t-x.T*self.w)
        self.num_obs += 1

In [79]:
test_size = 1000
# Test function
f = lambda x: 0.07*x**3-1.9*x**2-7.9*x+13
y = np.array([f(i) for i in range(test_size)])
noise = np.random.randn(test_size)
noisy_y = y + noise

num_vars = 4
lam = 0.98
LS_2 = RLS_2(num_vars,lam,1)

In [80]:
pred_x = []
pred_y = []

for i in range(200):
    x = np.matrix(np.zeros((1,num_vars)))
    for j in range(num_vars):
        x[0,j] = i**j 
    pred_x.append(i)
    pred_y.append(float(x*LS_2.w))
    LS_2.add_obs(x.T,y[i])

    if(i % 20 == 0):
        print("Weigths at trail [{}]:".format(LS_2.num_obs -1), LS_2.w.T)

Weigths at trail [0]: [[6.56565657 0.         0.         0.        ]]
Weigths at trail [20]: [[ 7.62106299 -5.83938845 -2.1048661   0.0758774 ]]
Weigths at trail [40]: [[ 9.3438164  -7.27051627 -1.9298864   0.0704192 ]]
Weigths at trail [60]: [[10.33431478 -7.60553426 -1.90910851  0.07008394]]
Weigths at trail [80]: [[10.97050614 -7.73665477 -1.90371865  0.07002537]]
Weigths at trail [100]: [[11.40501376 -7.80006635 -1.9017874   0.07000963]]
Weigths at trail [120]: [[11.7157714  -7.83472061 -1.90095576  0.07000424]]
Weigths at trail [140]: [[11.94646653 -7.85529375 -1.90055122  0.07000207]]
Weigths at trail [160]: [[12.1230334  -7.86827123 -1.9003364   0.07000109]]
Weigths at trail [180]: [[12.26160939 -7.87684753 -1.90021449  0.07000061]]
